# Named Entity Recognition (NER)

Named Entity Recognition (NER) will help us computationally identify people, places, and things (of various kinds) in a text or collection of texts. It is useful for extracting key information from texts. You might use NER to identify the most frequently appearing characters in a novel or build a network of characters (related to network analysis), or you might use NER to identify the geographic locations mentioned in texts, a first step toward mapping the locations (related to spatial analysis).

In [1]:
# Install spaCy for NER and other Natural Language Processing (NLP) tasks
!pip install -U spacy

     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
      --------------------------------------- 0.2/14.9 MB 6.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/14.9 MB 15.4 MB/s eta 0:00:01
     ------ --------------------------------- 2.4/14.9 MB 19.1 MB/s eta 0:00:01
     ----------- ---------------------------- 4.3/14.9 MB 24.9 MB/s eta 0:00:01
     ------------------- -------------------- 7.2/14.9 MB 33.0 MB/s eta 0:00:01
     --------------------------- ----------- 10.6/14.9 MB 43.7 MB/s eta 0:00:01
     ------------------------------------ -- 14.1/14.9 MB 65.6 MB/s eta 0:00:01
     --------------------------------------  14.9/14.9 MB 73.1 MB/s eta 0:00:01
     --------------------------------------- 14.9/14.9 MB 54.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------- ------------------------------ 3.0/12.9 MB 92.5 MB/s eta 0:00:01
     -------------------- ------------------- 6.5/12.9 MB


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


spaCy relies on machine learning models that were trained on a large amount of carefully-labeled texts. The English-language spaCy model that we’re going to use was trained on an annotated corpus called "OntoNotes": 2 million+ words drawn from "news, broadcast, talk shows, weblogs, usenet newsgroups, and conversational telephone speech," which were meticulously tagged by a group of researchers and professionals for people’s names and places, for nouns and verbs, for subjects and objects, and much more.

In [2]:
import spacy
from spacy import displacy # visualization based on spaCy
from collections import Counter # counting the results
import pandas as pd # dealing with dataframe

ImportError: cannot import name 'Sentinel' from 'typing_extensions' (C:\Users\colto\AppData\Roaming\Python\Python310\site-packages\typing_extensions.py)

In [3]:
# Download the English-language model
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB 6.3 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 24.7 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 42.3 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.8 MB 42.9 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 65.6 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 59.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Load the English language model
import en_core_web_sm
nlp = en_core_web_sm.load()

ImportError: cannot import name 'Sentinel' from 'typing_extensions' (C:\Users\colto\AppData\Roaming\Python\Python310\site-packages\typing_extensions.py)

In [5]:
# Load the dataset
movie = pd.read_csv('Data/movie_dialogue.csv')
movie

NameError: name 'pd' is not defined

In [6]:
# Let's see the lines of the 1st character in the 1st movie
print(movie['lines'][0])

They do not! / I hope so. / Let's go. / Okay -- you're gonna need to learn how to lie. / I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit? / Like my fear of wearing pastels? / What good stuff? / Me.  This endless ...blonde babble. I'm like, boring myself. / do you listen to this crap? / Then Guillermo says, "If you go any lighter, you're gonna look like an extra on 90210." / But / Well, no... / I was? / Tons / You know Chastity? / Hi. / Who knows?  All I've ever heard her say is that she'd dip before dating a guy that smokes. / Lesbian?  No. I found a picture of Jared Leto in one of her drawers, so I'm pretty sure she's not harboring same-sex tendencies. / I really, really, really wanna go, but I can't.  Not unless my sister goes. / Eber's Deep Conditioner every two days. And I never, ever use a blowdryer without the diffuser attachment. / You're sweet. / I counted on you to help my cause. You and that thug are obviously failing. Aren

In [7]:
# Visualize the name entity spaCy recognizes (note that the results are not perfect)
document = nlp(movie['lines'][0])
displacy.render(document, style="ent")

# Exploring Different Types of Entities

In [8]:
# Count and sort the number of characters with lines of each movie
movie.groupby('mname')['mid'].count().sort_values(ascending=False)

mname
magnolia                          18
lone star                         16
the anniversary party             12
nixon                             12
grand hotel                       11
                                  ..
dark city                          1
quantum project                    1
the nightmare before christmas     1
metropolis                         1
predator                           1
Name: mid, Length: 600, dtype: int64

In [9]:
# Count the number of entities in the lines of the characters in the movie magnolia
ent_types = dict() # initialize a dictionary
for line in movie[movie['mname']=='magnolia']['lines']: # loop through the lines in "magnolia"
    doc = nlp(line)
    for entity in doc.ents: # for each character, loop through all the entities
        label = entity.label_ # get their labels
        if label not in ent_types: # make sure there's a key for this label in the dictionary
            ent_types[label] = Counter() # each label key points to a Counter for examples
        text = entity.text
        ent_types[label][text] += 1 # count the number of times we see each example

In [10]:
# Count of each type of entities
for etype, examples in ent_types.items():
    print(etype, len(examples))

PERSON 73
GPE 12
ORG 31
TIME 11
CARDINAL 19
DATE 27
NORP 2
WORK_OF_ART 8
ORDINAL 1
QUANTITY 3
LAW 1
FAC 2


In [11]:
# Explain the entity type "PERSON"
spacy.explain('PERSON')

'People, including fictional'

In [12]:
# Get all people
people = []
for line in movie[movie['mname']=='magnolia']['lines']:
    doc = nlp(line)
    for named_entity in doc.ents:
        if named_entity.label_ == "PERSON": # we only want the labels with a type "PERSON"
            people.append(named_entity.text)
people_count = Counter(people)
# sort the people names by their occurences, and convert the results into a dataframe
people_magnolia = pd.DataFrame(people_count.most_common(), columns=['character', 'count'])
people_magnolia

,character,count
0,Linda,15
1,Frank,14
2,Phil,7
3,Earl,6
4,Jimmy,6
...,...,...
68,I'M SMART,1
69,Jean Baptiste,1
70,Willa Cather,1
71,Dad,1


In [13]:
# Explain a certain type of entity "TIME" and "DATE"
print("TIME:", spacy.explain('TIME'))
print("DATE:", spacy.explain('DATE'))

TIME: Times smaller than a day
DATE: Absolute or relative dates or periods


# Task 1

In [14]:
# Get entities related to time and their numbers in the movie magnolia and convert the sorted results to a dataframe.
# Like what we did for "people", first create an empty list, and then loop through each line in the movie
# and use "nlp" to convert the line. Next, loop through each entity of the line, and select the entity based on labels
# "TIME" or "DATE", and append the result to the empity list. Finally, use Counter to count the results,
# create a dataframe and sort them using "most_common."

time = []
for line in movie[movie['mname']=='magnolia']['lines']:
    doc = nlp(line)
    for named_entity in doc.ents:
        if named_entity.label_ == "TIME" or named_entity.label_ == "DATE":
            time.append(named_entity.text)
time_count = Counter(time)
time_magnolia = pd.DataFrame(time_count.most_common(), columns=['character', 'count'])
print(time_magnolia)




                  character  count
0                     today      6
1                   tonight      4
2                last night      3
3                four years      2
4               ten o'clock      2
5                   an hour      2
6                  a minute      1
7             Eight o'clock      1
8       About two years ago      1
9                  tomorrow      1
10             seven-eleven      1
11               age twelve      1
12            age seventeen      1
13       thirty eight years      1
14             12 years old      1
15                      '84      1
16               one minute      1
17               five years      1
18                     1980      1
19          three years ago      1
20              Ten o'clock      1
21                    night      1
22           two and a half      1
23                    a day      1
24        about three years      1
25             thirty years      1
26                  ' years      1
27         twenty ye

In [15]:
# Get the total word count of all lines of movies released in the year of 1960
movie_1960_df = movie[(movie['year']==1960)]
movie_1960_wordcount = movie_1960_df["wordcount"].sum()
movie_1960_wordcount

19535

In [16]:
# Get the total word count of all lines of movies released in the year of 2009
movie_2009_df = movie[(movie['year']==2009)]
movie_2009_wordcount = movie_2009_df["wordcount"].sum()
movie_2009_wordcount

15858

# Task 2

In [17]:
# Calculate the number of entities related to time in movies released in 1960 and 2009, and divide by respective word counts.
# First, initialize a variable "time_1960_count" and set it as 0. Then like what we did before, loop through each line in 
# movie_1960_df['lines'], and use "nlp" to convert the line. Next, loop through each entity of the line, 
# and if the entity has a label of "TIME" or "DATE", add 1 to time_1960_count. Finally, print time_1960_count,
# divided by movie_1960_wordcount. And then repeat the process for 2009 movies.

time_1960_count = 0
for line in movie[movie['year']==1960]['lines']:
    doc = nlp(line)
    for named_entity in doc.ents:
        if named_entity.label_ == "TIME" or named_entity.label_ == "DATE":
            time_1960_count += 1 
print('1960:',time_1960_count/movie_1960_wordcount)

time_2009_count = 0
for line in movie[movie['year']==2009]['lines']:
    doc = nlp(line)
    for named_entity in doc.ents:
        if named_entity.label_ == "TIME" or named_entity.label_ == "DATE":
            time_2009_count += 1 
print('2009:', time_2009_count/movie_2009_wordcount)




1960: 0.008395188123880215
2009: 0.004540295119182747


# Part-of-Speech (POS) Tagging

Parts of speech are the grammatical units of language, such as (in English) nouns, verbs, adjectives, adverbs, pronouns, and prepositions. Each of these parts of speech plays a different role in a sentence. By computationally identifying parts of speech, we can start computationally exploring syntax, the relationship between words, rather than only focusing on words in isolation, as we did with tf-idf.

In [18]:
# Get the POS tagging of a sample text
sample = """Or set upon a golden bough to sing to lords and ladies of Byzantium of what is past, or passing, or to come."""
# This is an excerpt from "Sailing to Byzantium" by the Irish poet W. B. Yeats
document = nlp(sample)
options = {"compact": True, "distance": 90, "color": "yellow", "bg": "black", "font": "Gill Sans"}
displacy.render(document, style="dep", options=options) # visualize it

In [19]:
# Get part-of-speech tags
for token in document:
    print(token.text, token.pos_, token.dep_) # pos_ means part-of-speech tags, and dep_ means dependency

Or CCONJ cc
set VERB ROOT
upon SCONJ prep
a DET det
golden ADJ amod
bough NOUN pobj
to PART aux
sing VERB advcl
to ADP prep
lords NOUN pobj
and CCONJ cc
ladies NOUN conj
of ADP prep
Byzantium PROPN pobj
of ADP prep
what PRON nsubj
is AUX pcomp
past ADJ acomp
, PUNCT punct
or CCONJ cc
passing VERB conj
, PUNCT punct
or CCONJ cc
to PART aux
come VERB conj
. PUNCT punct


In [20]:
# Get verbs from the movie "magnolia" in the movie dialogue dataset

verbs = []
for line in movie[movie['mname']=='magnolia']['lines']:
    doc = nlp(line)
    for token in doc: # loop through the token, instead of entities
        if token.pos_ == 'VERB': # we only want the tokens with a POS tagging of "VERB"
            verbs.append(token.text)
verbs_count = Counter(verbs)
# sort the verb by their occurences, and convert the results into a dataframe
verbs_magnolia = pd.DataFrame(verbs_count.most_common(), columns=['verb', 'count'])
verbs_magnolia

,verb,count
0,know,80
1,have,79
2,do,75
3,go,50
4,want,49
...,...,...
460,drink,1
461,presume,1
462,spoken,1
463,Sounds,1


# Task 3

In [21]:
# Get the top 10 adjectives by count in movies released in 1960 and 2009.
# Like what we did for the movie "magnolia", first create an empty list of adjectives, and then loop through each line in 
# movie_1960_df['lines'], and use "nlp" to convert the line. Next, loop through each token of the line, and select the entity 
# based on POS tagging "ADJ", and append the result to the empity list. Finally, use Counter to count the results,
# create a dataframe and sort them using "most_common," and print the top 10 adjectives.
# And then repeat the process for 2009 movies.

adjectives = []
for line in movie[movie['year']==1960]['lines']:
    doc = nlp(line)
    for token in doc:
        if token.pos_ == 'ADJ':
            adjectives.append(token.text)
adj_count = Counter(adjectives)
adj_1960 = pd.DataFrame(adj_count.most_common(), columns=['adjectives', 'count'])
print('1960', adj_1960[:10])

adjectives2 = []
for line in movie[movie['year']==2009]['lines']:
    doc = nlp(line)
    for token in doc:
        if token.pos_ == 'ADJ':
            adjectives2.append(token.text)
adj2_count = Counter(adjectives2)
adj_2009 = pd.DataFrame(adj2_count.most_common(), columns=['adjectives', 'count'])
print('2009', adj_2009[:10])



1960   adjectives  count
0     little     35
1      right     26
2      sorry     23
3      wrong     23
4       good     20
5      other     19
6        old     18
7       Good     17
8       sure     17
9       wise     16
2009   adjectives  count
0       more     22
1       dead     19
2       good     19
3     little     17
4      other     15
5      right     15
6      ready     12
7       Good     11
8        bad     11
9       next     10


# Keyword Extraction

In [22]:
import re # regular expression
from IPython.display import Markdown, display # for visualization

In [23]:
# Visualize the data from the movie Casablanca
movie_casablanca = movie[movie['mname']=='casablanca'].reset_index(drop=True)
for line in movie_casablanca['lines']:
    displacy.render(nlp(line), style="ent")

In [24]:
# Define a function to find keywords in its context
def find_sentences_with_keyword(keyword, document): 
    # loop through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        sentence = sentence.text        
        # check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():            
            # use regular expression to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)            
            display(Markdown(sentence))

In [25]:
# Highlight the name of the protagonist Rick in its context of the lines of the second character of the movie
find_sentences_with_keyword(keyword="Rick", document=nlp(movie_casablanca['lines'][1]))

**Rick**'s wouldn't be **Rick**'s without them.

/ **Rick**, don't be a fool.

**Rick**, I'll put my cards on the table.

My dear **Rick**, when will you realize that in this world today isolationism is no longer a practical policy?

Hello, **Rick**. /

He is a difficult customer, that **Rick**.

Not for sure, Monsieur, but I will venture to guess that Ugarte left those letters with Monsieur **Rick**.

In [26]:
# Create a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in nlp(movie_casablanca['lines'][1]) if token.is_alpha]
tokens_and_labels

[('I', 'PRON'),
 ('shall', 'AUX'),
 ('remember', 'VERB'),
 ('to', 'PART'),
 ('pay', 'VERB'),
 ('it', 'PRON'),
 ('to', 'ADP'),
 ('myself', 'PRON'),
 ('Of', 'ADV'),
 ('course', 'ADV'),
 ('they', 'PRON'),
 ('stay', 'VERB'),
 ('Rick', 'PROPN'),
 ('would', 'AUX'),
 ('be', 'AUX'),
 ('Rick', 'PROPN'),
 ('without', 'ADP'),
 ('them', 'PRON'),
 ('Hmmm', 'PROPN'),
 ('I', 'PRON'),
 ('happen', 'VERB'),
 ('to', 'PART'),
 ('know', 'VERB'),
 ('that', 'SCONJ'),
 ('he', 'PRON'),
 ('gets', 'VERB'),
 ('ten', 'NUM'),
 ('percent', 'NOUN'),
 ('But', 'CCONJ'),
 ('he', 'PRON'),
 ('worth', 'ADJ'),
 ('five', 'NUM'),
 ('Ah', 'INTJ'),
 ('to', 'PART'),
 ('get', 'VERB'),
 ('out', 'ADP'),
 ('of', 'ADP'),
 ('Casablanca', 'PROPN'),
 ('and', 'CCONJ'),
 ('go', 'VERB'),
 ('to', 'ADP'),
 ('America', 'PROPN'),
 ('You', 'PRON'),
 ('a', 'DET'),
 ('lucky', 'ADJ'),
 ('man', 'NOUN'),
 ('Shall', 'AUX'),
 ('we', 'PRON'),
 ('draw', 'VERB'),
 ('up', 'ADP'),
 ('the', 'DET'),
 ('papers', 'NOUN'),
 ('or', 'CCONJ'),
 ('is', 'AUX'),
 ('o

In [27]:
# Define a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):  
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)   
    # loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):       
        # index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]        
        # append this word combo to the master list "ngrams"
        ngrams.append(ngram)        
    return ngrams

In [28]:
# Getting all bigram of the lines, including both the word and its POS label
bigrams = get_bigrams(tokens_and_labels)
bigrams

[[('I', 'PRON'), ('shall', 'AUX')],
 [('shall', 'AUX'), ('remember', 'VERB')],
 [('remember', 'VERB'), ('to', 'PART')],
 [('to', 'PART'), ('pay', 'VERB')],
 [('pay', 'VERB'), ('it', 'PRON')],
 [('it', 'PRON'), ('to', 'ADP')],
 [('to', 'ADP'), ('myself', 'PRON')],
 [('myself', 'PRON'), ('Of', 'ADV')],
 [('Of', 'ADV'), ('course', 'ADV')],
 [('course', 'ADV'), ('they', 'PRON')],
 [('they', 'PRON'), ('stay', 'VERB')],
 [('stay', 'VERB'), ('Rick', 'PROPN')],
 [('Rick', 'PROPN'), ('would', 'AUX')],
 [('would', 'AUX'), ('be', 'AUX')],
 [('be', 'AUX'), ('Rick', 'PROPN')],
 [('Rick', 'PROPN'), ('without', 'ADP')],
 [('without', 'ADP'), ('them', 'PRON')],
 [('them', 'PRON'), ('Hmmm', 'PROPN')],
 [('Hmmm', 'PROPN'), ('I', 'PRON')],
 [('I', 'PRON'), ('happen', 'VERB')],
 [('happen', 'VERB'), ('to', 'PART')],
 [('to', 'PART'), ('know', 'VERB')],
 [('know', 'VERB'), ('that', 'SCONJ')],
 [('that', 'SCONJ'), ('he', 'PRON')],
 [('he', 'PRON'), ('gets', 'VERB')],
 [('gets', 'VERB'), ('ten', 'NUM')],
 [(

In [29]:
# Define a function to get the neighboring words based on bigrams
def get_neighbor_words(keyword, bigrams, pos_label = None):    
    neighbor_words = []
    keyword = keyword.lower()    
    for bigram in bigrams:       
        # extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]                
        # check to see if keyword is in the bigram
        if keyword in words:         
            for word, label in bigram:
                if word.lower() != keyword: # focusing on the neighbor word, not the keyword
                    neighbor_words.append(word.lower())    
    # return the word list after sorting it
    return Counter(neighbor_words).most_common()

In [30]:
# Get the neighboring words of the character Rick
get_neighbor_words("Rick", bigrams)

[('stay', 1),
 ('would', 1),
 ('be', 1),
 ('without', 1),
 ('enough', 1),
 ('do', 1),
 ('partner', 1),
 ('i', 1),
 ('dear', 1),
 ('when', 1),
 ('hello', 1),
 ('it', 1),
 ('that', 1),
 ('one', 1),
 ('monsieur', 1),
 ('those', 1)]

# Task 4

In [31]:
# Print the neighboring words of the character Rick in each character's lines of the movie Casablanca and the characters' names.
# Write a loop through the movie_casablanca dataframe, copy the codes above of creating a list of tokens and POS labels 
# from each line if the token is a word, and then get all bigram of the lines. Finally, print movie_casablanca['cname'] of 
# that line, along with the neighboring words of Rick in that line.

for i in range(len(movie_casablanca['lines'])):
    tokens_and_labels = [(token.text, token.pos_) for token in nlp(movie_casablanca['lines'][i]) if token.is_alpha]
    bigrams = get_bigrams(tokens_and_labels)
    print(movie_casablanca['cname'][i], get_neighbor_words('Rick',bigrams))


ANNINA [('monsieur', 3), ('i', 1), ('what', 1)]
FERRARI [('stay', 1), ('would', 1), ('be', 1), ('without', 1), ('enough', 1), ('do', 1), ('partner', 1), ('i', 1), ('dear', 1), ('when', 1), ('hello', 1), ('it', 1), ('that', 1), ('one', 1), ('monsieur', 1), ('those', 1)]
ILSA [('no', 3), ('i', 3), ('is', 2), ('what', 2), ('oh', 1), ('but', 1), ('the', 1), ('will', 1), ('yes', 1), ('story', 1), ('do', 1), ('me', 1), ('it', 1), ('not', 1), ('some', 1), ('goodbye', 1), ('god', 1), ('about', 1), ('with', 1), ('he', 1), ('hello', 1), ('who', 1)]
LASZLO [('day', 1), ('do', 1), ('about', 1), ('in', 1)]
RENAULT [('and', 3), ('in', 2), ('with', 2), ('is', 2), ('about', 2), ('you', 2), ('there', 2), ('earlier', 1), ('met', 1), ('mademoiselle', 1), ('but', 1), ('sam', 1), ('monsieur', 1), ('victor', 1), ('if', 1), ('has', 1), ('of', 1), ('courage', 1), ('comes', 1), ('this', 1), ('at', 1), ('everybody', 1), ('to', 1), ('realizing', 1), ('well', 1), ('later', 1), ('huh', 1), ('germans', 1), ('have',

# Task 5

What insights can be derived from the movie dialogue dataset using the Named Entity Recognition extraction method? Similarly, what insights can be gained through the Position-of-Speech tagging method? Please provide an example for each method, either based on the experiments covered above or by thinking of something else.

Some of the key takeaways we can pull from the movie dialogues using the Named entity recognition is being able to identify different types of entities in the movie dialogue. Although not 100% accurate this does serve to get an idea of how many People, Time, Cardinal, etc are in a given text. For instance if we are doing an analysis of the movie magnolia like in Task 1 but analyzing the motif of time we can analyze the occurrences of different words like today, tonight, years, etc. By doing this we can extract different themes from the text/movie. With position-of-speech tagging we can understand what common words are surrounding characters from a piece of work like a movie/text/etc. What we did in Task 4 is show different characters names with neighboring words. This can give a lot of context for the characters and their importance/role in the play such as Annina being referred to as 'Monsieur' multiple times showing that she is a woman of high importance maybe even high social status or class. 